In [3]:
# Import Butler 
from lsst.eo_utils.base import butler_utils

In [4]:
# Set the path to the Butler repository and import a Butler
butler = butler_utils.get_butler_by_repo('bot')

In [5]:
# Make a generic data ID that gets everything about run 6106D
dd = dict(run='6787D')

In [6]:
# Get the list of visits and print its length
visits = butler.queryMetadata('raw', 'visit', run='6787D')
print("Total visits:", len(visits))

Total visits: 0


In [13]:
# Get possible values for some of the metadata
# Note that  queryMetadata returns the list of all possible values 
# for a meta data files for a dataId
print("Metadata values")
for key in ['raftName', 'detectorName', 'testType', 'imageType']:
    vals = butler.queryMetadata('raw', key, dd)
    print ("%20s" % key, vals)
print("\n\n")

Metadata values
            raftName []
        detectorName []
            testType []
           imageType []





In [6]:
# A function to get dataIds and sort them 
# Note that we are refining the dataId to only 
# get the possible values for a given testType and imageType
def sort_data_ids(run):
    dd = dict(run=run)
    dataIdDict = {}
    slots = butler.queryMetadata('raw', 'detectorName', dd)

    for slot in slots:
        dd['detectorName'] = slot
        t_types = butler.queryMetadata('raw', 'testType', dd)
        ttype_dict = {}
        for test_type in t_types:
            dd['testType'] = test_type
            i_types = butler.queryMetadata('raw', 'imageType', dd)
            itype_dict = {}
            for i_type in i_types:
                dd['imageType'] = i_type
                ss = butler.subset('raw', '', dd)
                itype_dict[i_type] = ss.cache
            ttype_dict[test_type] = itype_dict
            dd.pop('imageType')
        dataIdDict[slot] = ttype_dict

    return dataIdDict

In [7]:
# A function to print how many exposure of each type we have
def count_exposures(ddict):
    for key, val in ddict.items():
        for key2, val2 in val.items():
            print("Test %s has %i %s exposures" % (key, len(val2), key2))
    print("\n\n")

In [8]:
# A simple function to print and OrderedDict
# This is useful to print the FITS header info
def print_dict(odict):
    for key, val in odict.items():
        print("%20s : %s" % (key, val))

In [9]:
# Ok, lets get some dataIds and print out the counts
ddict = sort_data_ids('6106D')
count_exposures(ddict['S00'])

OperationalError: no such column: expId

In [ ]:
# Get list of dataIds for FLAT testtype and FLAT imagetype
flats = ddict['S00']['FLAT']['FLAT']

In [ ]:
# Note that we could also do this
dataId = dict(run='6106D', detectorName='S00', testType='FLAT', imageType='FLAT')
flats_check = butler.subset('raw', '', dataId).cache


In [ ]:
# Get a flat exposure
aFlat = butler.get('raw', flats[0])

# Now we print out some information for that exposure
print("Info for one flat exposure")
print("  Exposure time %.3fs" % aFlat.getInfo().getVisitInfo().getExposureTime())
print("  Filter %s" % aFlat.getFilter().getName())
print("  Metadata: ")
print_dict(aFlat.getInfo().getMetadata().toOrderedDict())
